## Imports and API

In [32]:
# Libraries
from openai import OpenAI
import requests
import feedparser
from dotenv import load_dotenv
import os

client = OpenAI()
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

## ArVix search agent

In [33]:
def search_arxiv(query, max_results=5):
    url = f'http://export.arxiv.org/api/query?search_query={query}&start=0&max_results={max_results}'
    feed = feedparser.parse(requests.get(url).text)
    
    results = []
    for entry in feed.entries:
        results.append({
            'title': entry.title,
            'summary': entry.summary,
            'link': entry.link
        })
    return results


## OpenAlex search agent

In [34]:
def search_openalex(query, max_results=5):
    url = f"https://api.openalex.org/works?search={query}&per-page={max_results}"
    response = requests.get(url)
    data = response.json()
    
    results = []
    for item in data.get('results', []):
        results.append({
            'title': item.get('display_name', 'No Title'),
            'summary': item.get('abstract_inverted_index', {}),
            'link': item.get('id', 'No Link')
        })
    return results


In [35]:
def merge_results(arxiv_results, openalex_results):
    return arxiv_results + openalex_results

## Summerizing agent

In [36]:
def summarize_papers(papers):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = f"Summarize the following research papers:\n{content}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


## Critic Agent

In [37]:
def critique_summary(summary_text):
    prompt = (
        "Review the following summary for bias, missing key points, or factual errors. "
        "Suggest improvements clearly:\n\n"
        f"{summary_text}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    critique = response.choices[0].message.content
    
    if any(keyword in critique.lower() for keyword in ["missing", "improve", "bias", "error"]):
        return critique, 'needs_rewrite'
    else:
        return critique, 'approved'


## Verifier agent

In [38]:
def verify_summary(summary_text):
    prompt = (
        "Fact-check the following summary. "
        "Highlight any factual inaccuracies or unsupported claims:\n\n"
        f"{summary_text}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [39]:
def compose_final_answer(summary, critique, verification):
    return f"Summary:\n{summary}\n\nCritique:\n{critique}\n\nVerification Notes:\n{verification}"

In [40]:
# Example Query
query = "latest advancements in AI alignment"

# Step 1: Search
arxiv_results = search_arxiv(query, max_results=3)
openalex_results = search_openalex(query, max_results=3)
merged_results = merge_results(arxiv_results, openalex_results)

# Step 2: Summarize
summary = summarize_papers(merged_results)

# Step 3: Critique with loop if needed
critique, status = critique_summary(summary)
if status == 'needs_rewrite':
    print("Critique suggests improvements. Re-summarizing...")
    summary = summarize_papers(merged_results)
    critique, status = critique_summary(summary)

# Step 4: Verify
verification = verify_summary(summary)

# Step 5: Final Answer
final_output = compose_final_answer(summary, critique, verification)
print(final_output)

Critique suggests improvements. Re-summarizing...
Summary:
Title: Innovating for Tomorrow: The Convergence of SE and Green AI
This paper discusses how advancements in machine learning are impacting software engineering processes, with a focus on creating environmentally friendly AI-enabled software systems.

Title: Reflection on Data Storytelling Tools in the Generative AI Era from the Human-AI Collaboration Perspective
The paper explores the use of generative AI techniques in data storytelling tools and analyzes collaboration patterns between humans and AI, highlighting opportunities for future innovation.

Title: WordDecipher: Enhancing Digital Workspace Communication with Explainable AI for Non-native English Speakers
This paper presents WordDecipher, an explainable AI tool designed to improve communication for non-native English speakers in digital workspaces by providing rewriting suggestions aligned with users' intentions.

Title: Big data in healthcare: management, analysis and 